In [1]:
library("forecast")
library("stringr")
library("ggplot2")
library("MLmetrics")

Warning message:
"package 'MLmetrics' was built under R version 3.5.2"
Attaching package: 'MLmetrics'

The following object is masked from 'package:base':

    Recall



In [2]:
setwd("C:\\Users\\kevin\\ziknet-trends-rolling\\data\\Brazil\\processed_data")

In [3]:
files = list.files(".", pattern="*.csv")
files

[1] "Bahia_2016-2017.csv"        "MatoGrosso_2016-2017.csv"  
[3] "MinasGerais_2016-2017.csv"  "RioDeJaneiro_2016-2017.csv"
[5] "SaoPaulo_2016-2017.csv"

In [4]:
myfiles = lapply(files, function(i){
    read.csv(i, header=TRUE)
})
print(myfiles)

[[1]]
          Date Searches        Cases
1   04/01/2016       20  316.1987434
2   11/01/2016       28 1053.2329140
3   18/01/2016       22 1790.2670860
4   25/01/2016       40 2527.3012570
5   01/02/2016       45 3264.3354280
6   08/02/2016       37 3985.9130660
7   15/02/2016       63 4614.7515070
8   22/02/2016       71 5228.1334150
9   29/02/2016       79 5611.7879930
10  07/03/2016       84 4617.0785920
11  14/03/2016       71 3392.6418620
12  21/03/2016       88 2601.3581380
13  28/03/2016       65 2243.2274200
14  04/04/2016       74 1883.7311340
15  11/04/2016       54 1516.0414460
16  18/04/2016       61 1146.9861900
17  25/04/2016       42  777.9309341
18  02/05/2016       35  546.9723721
19  09/05/2016       35  454.1105040
20  16/05/2016       74  361.2486358
21  23/05/2016       38  268.3867677
22  30/05/2016       46  175.5248996
23  06/06/2016       29  134.4892767
24  13/06/2016       27  145.2798989
25  20/06/2016       22  156.0705212
26  27/06/2016       32  166.861

In [5]:
setwd("C:\\Users\\kevin\\ziknet-trends-rolling\\Brazil\\SARIMA\\2Week")

In [6]:
myplot <- function(dataset, filename) {
    print("-----")
    print(filename)
    split_index = nrow(dataset)/2
    
    window_size = 4
    lag_weeks = 2

    train = dataset[1:(split_index - lag_weeks + 1),2:3]
    test = dataset[(split_index+1):(nrow(dataset)), 2:3]

    endIndex = nrow(test)
    trainIndex = nrow(train)
    
    predictions = numeric(0)

    for (index in 1:endIndex){
        fit <- auto.arima(dataset$Cases[1:trainIndex+index-1], seasonal=TRUE, xreg=dataset$Searches[1:trainIndex+index-1])
        
        #Forecast regressors
        fit_reg = auto.arima(dataset$Searches[1:trainIndex+index-1])
        reg_forecast <- forecast(fit_reg, h=lag_weeks)
        reg_forecast_frame <- data.frame(reg_forecast)
        reg_forecast_point_forecast <- reg_forecast_frame$Point.Forecast
        reg_df = dataset$Searches[1:trainIndex+index-1]
        l = 1
        reg_length = length(reg_df)
        
        while(l <= lag_weeks) {
            reg_df[reg_length + l] = reg_df[l]
            l = l + 1
        }
        
        #Forecast
        
#         print(arimaorder(fit))
        fcast <- forecast(fit, h=lag_weeks, xreg = reg_df)
        forecasted_data <- data.frame(fcast)
        predictions[index] <- forecasted_data$Point.Forecast[lag_weeks]
    }
    
    length(predictions)
    print(filename)
#     output = cat("1Week", filename)
    write.csv(predictions, file=filename)
    mse = MSE(predictions, test$Cases)
    print(mse)
    
}

In [7]:
fileIndex = 1
for(file in myfiles) {
    myplot(file, files[fileIndex])
    fileIndex = fileIndex + 1
}

[1] "-----"
[1] "Bahia_2016-2017.csv"
[1] "Bahia_2016-2017.csv"
[1] 1969.536
[1] "-----"
[1] "MatoGrosso_2016-2017.csv"
[1] "MatoGrosso_2016-2017.csv"
[1] 2676.196
[1] "-----"
[1] "MinasGerais_2016-2017.csv"
[1] "MinasGerais_2016-2017.csv"
[1] 397.9062
[1] "-----"
[1] "RioDeJaneiro_2016-2017.csv"
[1] "RioDeJaneiro_2016-2017.csv"
[1] 6604.449
[1] "-----"
[1] "SaoPaulo_2016-2017.csv"
[1] "SaoPaulo_2016-2017.csv"
[1] 140.3852
